In [6]:
from astropy.table import Table
import scipy.spatial.distance as dist
from astropy import units as u
from astropy.cosmology import Planck13
from astropy.io import fits
from astropy.coordinates import SkyCoord
import sptpol_software as sps
from sptpol_software.util.tools import stat
from sptpol_software.observation import *
import sptpol_software.observation as obs
import sptpol_software as sps
import sptpol_software.observation.sky
from sptpol_software.util import files
import PIL
import pickle
from astropy import constants as ap_const
import scipy.constants as const
from scipy.integrate import quad
import matplotlib.pyplot as plt
import pandas as pd
import glob
import astropy as ap
import scipy as sp
import numpy as np
import os
import time 
import pdb
os.chdir("/home/mitchell/Documents/masters/masters/scripts/")
import cmb
import galaxy_pairs
#data_location = input("Enter Path Location of data")
os.chdir("/home/mitchell/Documents/masters/masters/data/")
cwd = os.getcwd()

import scipy.ndimage
cosmo = Planck13

pywtl packages could not be imported.  This may break plotting code.
FlatLambdaCDM(name="Planck15", H0=67.7 km / (Mpc s), Om0=0.307, Tcmb0=2.725 K, Neff=3.05, m_nu=[0.   0.   0.06] eV, Ob0=0.0486)


In [7]:
df = pd.read_pickle('DES_cat.pkl')


In [8]:
print("Computing Comoving Distances from Redshifts in Galaxy Catalogue ...")
start = time.time()
df['COMOVING'] = pd.Series(
    cosmo.comoving_distance(df['ZREDMAGIC']))
df['COMOVING_E'] = pd.Series(
    cosmo.comoving_distance(df['ZREDMAGIC_E']))

Computing Comoving Distances from Redshifts in Galaxy Catalogue ...


In [9]:
norm_ra = []
for i in range(len(df)):
    norm_ra.append(galaxy_pairs.normalize(df['RA'][i],-180,180))
df['NORM_RA'] = pd.Series(norm_ra)


In [10]:
ra_range = np.array([-32,32])
dec_range = np.array([-52,-67])


In [11]:
cut_df = df[((df.DEC < dec_range[0]) & (df.DEC > dec_range[1]))
            & ((df.NORM_RA > ra_range[0]) & (df.NORM_RA < ra_range[1]))]
cut_df = cut_df.reset_index(drop=True)


In [12]:
sky_locs = np.asarray([cut_df['NORM_RA'].as_matrix().T,cut_df['DEC'].as_matrix().T]).T

frame_tree = scipy.spatial.cKDTree(sky_locs, leafsize=100)

result_set = frame_tree.query_pairs(r=0.4)

result_list = list(result_set)

result_array = np.asarray(result_list)

cut_pairs_df = pd.DataFrame(data=result_array,columns = ['galaxy_index_1','galaxy_index_2'])

In [13]:
cut_pairs_df['galaxy_index_1'] = cut_pairs_df.galaxy_index_1.astype(int)
cut_pairs_df['galaxy_index_2'] = cut_pairs_df.galaxy_index_2.astype(int)

print("Num of Pairs Produced on Footprint = " + str(len(cut_pairs_df)))

Num of Pairs Produced on Footprint = 6021580


In [14]:
gal_1_comov = cut_df['COMOVING'].iloc[cut_pairs_df['galaxy_index_1']]
gal_2_comov = cut_df['COMOVING'].iloc[cut_pairs_df['galaxy_index_2']]
comov_sep = pd.Series(abs(gal_1_comov.values - gal_2_comov.values))

In [15]:
cut_pairs_df['SEP_LOS'] = comov_sep


min_los = 100/cosmo.h
max_los = 200/cosmo.h


cut_pairs_df = cut_pairs_df[(cut_pairs_df.SEP_LOS < max_los) & (cut_pairs_df.SEP_LOS > min_los)]

cut_pairs_df = cut_pairs_df.reset_index()
print("Num of Pairs after LOS Cuts = " + str(len(cut_pairs_df)))

Num of Pairs after LOS Cuts = 886848


In [ ]:
gal_1_z = cut_df['ZREDMAGIC'].iloc[cut_pairs_df['galaxy_index_1']]
gal_2_z = cut_df['ZREDMAGIC'].iloc[cut_pairs_df['galaxy_index_2']]

print("Appending Redshifts of Pair to Dataframe...")
cut_pairs_df['Z1'] = pd.Series(gal_1_z.values)
cut_pairs_df['Z2'] = pd.Series(gal_2_z.values)

end = time.time()
print("Done in " + str(end-start) + " seconds")
start = end

print("Computing Average Redshift...")
avg_redshift = pd.Series(np.mean([gal_1_z,gal_2_z],axis=0))

end = time.time()
print("Done in " + str(end-start) + " seconds")
start = end

print("Computing Transverse Comoving Distance at this redshift...")
transverse_comoving = cosmo.comoving_transverse_distance(avg_redshift)

cut_pairs_df['COMOV_TRV'] = transverse_comoving

end = time.time()
print("Done in " + str(end-start) + " seconds")
start = end

print("Appending (RA,DEC) to Dataframe...")
cut_pairs_df['RA_1'] = pd.Series(cut_df['RA'].iloc[cut_pairs_df['galaxy_index_1']].values)
cut_pairs_df['RA_2'] = pd.Series(cut_df['RA'].iloc[cut_pairs_df['galaxy_index_2']].values)
cut_pairs_df['DEC_1'] = pd.Series(cut_df['DEC'].iloc[cut_pairs_df['galaxy_index_1']].values)
cut_pairs_df['DEC_2'] = pd.Series(cut_df['DEC'].iloc[cut_pairs_df['galaxy_index_2']].values)

end = time.time()
print("Done in " + str(end-start) + " seconds")
start = end

print("Storing as Sky Coordinates...")
coord_1 = SkyCoord(ra= cut_pairs_df['RA_1']*u.degree,dec = cut_pairs_df['DEC_1']*u.degree)
coord_2 = SkyCoord(ra= cut_pairs_df['RA_2']*u.degree,dec = cut_pairs_df['DEC_2']*u.degree)

end = time.time()
print("Done in " + str(end-start) + " seconds")
start = end


Appending Redshifts of Pair to Dataframe...
Done in 276.11403203 seconds
Computing Average Redshift...
Done in 0.352884054184 seconds
Computing Transverse Comoving Distance at this redshift...
Done in 12.0261898041 seconds
Appending (RA,DEC) to Dataframe...
Done in 1.45157408714 seconds
Storing as Sky Coordinates...
